In [1]:
import pickle, random, math
import numpy as np
from sklearn.isotonic import IsotonicRegression
import traj_dist.distance as tdist

In [2]:
indices = np.arange(10000)

In [3]:
with open('cellGps', 'rb') as f:
    cellgps = pickle.load(f)
cellgps = np.array([[cellgps[i]] for i in range(18862)])

In [4]:
with open('testgps', 'rb') as f:
    gps = pickle.load(f)        # 后面画图的时候用到了这个gps变量，这里还不能动

gpsp = np.array(gps)[indices]


/tmp/ipykernel_1479110/1414089803.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  gpsp = np.array(gps)[indices]


In [5]:
# landmarks = np.array([6961, 2134, 3805, 6813, 2615, 3879, 3342, 1574])
landmarks = np.array([520, 1607, 5819, 4532, 4865, 2892, 4064, 3132, 6772,  111])
# landmarks = np.array([520, 1607, 5819, 4532, 4865, 2892, 4064, 3132, 6772,  111, 6961, 2134, 3805, 6813, 2615, 3879, 3342, 1574])
# 这个大landmark尝试过了，效果很差
landmarks

array([ 520, 1607, 5819, 4532, 4865, 2892, 4064, 3132, 6772,  111])

In [6]:
with open('128testEmbeddingNeutraj25', 'rb') as f:
    embed = pickle.load(f)
embedp = np.array(embed)[indices]


# # Compute distance matrix using discrete frechet distance
# dist_embed = np.zeros((len(embedp), len(embedp)))
# for i in range(len(embedp)):
#     for j in range(i, len(embedp)):
#         a, b = embedp[i], embedp[j]
#         distance = np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
#         dist_embed[i, j] = distance
#         dist_embed[j, i] = distance
# with open('dist_embed', 'wb') as f:
#     pickle.dump(dist_embed_128, f)

with open('dist_embed_huge_128', 'rb') as f:
    dist_embed = pickle.load(f)

In [7]:
def similarity(a, b):
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [8]:
def point2point(p1, p2):
    '''
    p = [lat, lon]
    '''
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

In [9]:
def point2traj(p, t):
    '''
    p=[[lat, lon]], t=[[lat1, lon2],..., [latn, lonn]] 
    '''
    p = p[0]
    return min([point2point(p, tt) for tt in t])

In [10]:
# compute the distance between landmarks
dist = []
simi = []

for i in landmarks:
    for j in landmarks:
        dist.append(tdist.hausdorff(gpsp[i], gpsp[j]))
        simi.append(similarity(embedp[i], embedp[j]))

dist = np.array(dist)
simi = np.array(simi)


In [11]:
# added by myself
print(np.shape(dist))
print(np.shape(simi))

(100,)
(100,)


In [12]:
unique_D = np.unique(dist)
unique_S = np.array([np.mean(simi[dist == x]) for x in unique_D])
sorted_indices = np.argsort(unique_D)
dist = unique_D[sorted_indices]
simi = unique_S[sorted_indices]

In [13]:
# added by myself
print(np.shape(dist))
print(np.shape(simi))
print(dist)
print(dist[::-1])

(46,)
(46,)
[0.         0.01304424 0.01585834 0.0168447  0.01697541 0.01880094
 0.02342602 0.02446219 0.02449195 0.02616535 0.0264929  0.02673071
 0.02876754 0.02877373 0.02891431 0.03071352 0.03095061 0.03136727
 0.03180297 0.03293044 0.03351794 0.03359774 0.03467561 0.0368337
 0.03693791 0.0377616  0.03839552 0.03846403 0.03910858 0.0395971
 0.04134674 0.04143431 0.04267498 0.04641589 0.05007574 0.05736948
 0.06279348 0.10686986 0.10990254 0.10998726 0.11235127 0.11245807
 0.11471026 0.11512442 0.11987171 0.12346684]
[0.12346684 0.11987171 0.11512442 0.11471026 0.11245807 0.11235127
 0.10998726 0.10990254 0.10686986 0.06279348 0.05736948 0.05007574
 0.04641589 0.04267498 0.04143431 0.04134674 0.0395971  0.03910858
 0.03846403 0.03839552 0.0377616  0.03693791 0.0368337  0.03467561
 0.03359774 0.03351794 0.03293044 0.03180297 0.03136727 0.03095061
 0.03071352 0.02891431 0.02877373 0.02876754 0.02673071 0.0264929
 0.02616535 0.02449195 0.02446219 0.02342602 0.01880094 0.01697541
 0.0168

In [14]:
print(dist[:10])
print(dist[::-1][:10])

[0.         0.01304424 0.01585834 0.0168447  0.01697541 0.01880094
 0.02342602 0.02446219 0.02449195 0.02616535]
[0.12346684 0.11987171 0.11512442 0.11471026 0.11245807 0.11235127
 0.10998726 0.10990254 0.10686986 0.06279348]


In [15]:
simi[:10]

array([1.        , 0.88403887, 0.84599376, 0.96050507, 0.95132214,
       0.93009245, 0.8760671 , 0.9185697 , 0.7536604 , 0.90636593],
      dtype=float32)

In [16]:
# 计算landmarks之间的相似性和距离用以训练保序回归模型得到距离和相似度之间的负相关关系
iso_reg = IsotonicRegression(increasing=False)
iso_reg.fit(dist, simi)

IsotonicRegression(increasing=False)

In [17]:
# dist_real = tdist.cdist(cellgps[:, :, [1,0]], gpsp[landmarks], 'haushorff')
# 计算每一个gpscell和每一个landmark之间的距离吧
dist_real = [[point2traj(cellgps[:, :, [1,0]][i], gpsp[landmarks][j]) for j in range(len(landmarks))] for i in range(18862)]

sim_real = [np.nan_to_num(iso_reg.predict(dist_real[i]), nan=0) for i in range(18862)]
coord_real = np.array(sim_real)

In [18]:
# 18862 个 cell 和锚定点的 距离和相似度
print(np.shape(dist_real))
print(np.shape(sim_real))

(18862, 10)
(18862, 10)


In [19]:
coord_embed = [[dist_embed[i, j] for j in landmarks] for i in range(10000)]
coord_embed = np.array(coord_embed)

In [20]:
topk = 10

In [21]:
# 为每一个testgps计算前topk相似的cellgps
# 我去，我今天才算真正理解了，这里我用的一直都是锚定轨迹，用锚定轨迹计算和各个cell的距离然后同样的办法计算每条轨迹
# 和每个地标轨迹之间的距离，然后取前几名，太6了
target = []
for i in range(10000):
    diff = coord_real - coord_embed[i]
    cost = np.array([np.sqrt(np.sum(diff[j]*diff[j])) for j in range(18862)])
    target.append(np.argsort(cost)[:topk])
    if i % 1000 == 0:
        print("done %d" % i)


done 0
done 1000
done 2000
done 3000
done 4000
done 5000
done 6000
done 7000
done 8000
done 9000


In [22]:
ave_dist = 0
dist_mat = []
for i in range(10000):
    for j in range(10):
        ave_dist += point2traj(cellgps[:, :, [1,0]][target[i][j]], gpsp[i])*111111
    # dist_mat.append([i, dist_real[i, target[i]]*111111])
ave_dist /= 10000 * 10
ave_dist

2959.223966656372

In [23]:
cellgps[:, :, [1,0]][target[1000]].shape

(10, 1, 2)

In [50]:
import folium
# choose = 3000    # default=3000
choose = np.random.randint(10000)
my_map = folium.Map(location = [gps[1000][0][1], gps[1000][-1][0]], zoom_start = 15, 
                    tiles='https://mt.google.com/vt/lyrs=m&x={x}&y={y}&z={z}', attr='default')#, tiles = 'https://{s}.tile-cyclosm.openstreetmap.fr/cyclosm/{z}/{x}/{y}.png', attr= 'default')
for t in [choose]:
    for i in range(len(gps[indices[t]])):
        folium.Circle( radius = 25,
                                location=[gps[indices[t]][i][1],gps[indices[t]][i][0]],    # 从原始数据的(纬度，经度)换成(经度，纬度)
                                color='blue',
                                opacity = 0.9,
                                fill=True,
                                fill_opacity = 0.9
                                ).add_to(my_map)
    folium.PolyLine(gps[indices[t]][:,[1,0]], color="blue",lineCap='round', weight=5, opacity=0.6).add_to(my_map)
for c in cellgps[:, :, [1,0]][target[choose]][:10,0,:]:    # 交换经度纬度，然后选择所选轨迹的前10个目标点.
    folium.Circle( radius = 25,
                                location=[c[1],c[0]],
                                color='red',
                                opacity = 0.9,
                                fill=True,
                                fill_opacity = 0.9
                                ).add_to(my_map)
my_map